In [1]:
import cv2
import numpy as np
import random

In [3]:
src=cv2.imread("lena.bmp",cv2.IMREAD_GRAYSCALE)

In [4]:
def generateGaussianNoise(image,amplitude):
    """
    image:Mat(2D-array)
    amplitude:noise intensity(int)
    output:Mat(2D-array)
    """
    rows,cols=image.shape
    noiseImg=np.zeros(shape=image.shape,dtype=image.dtype)
    for i in range(rows):
        for j in range(cols):
            noiseImg[i,j]=image[i,j]+amplitude*random.gauss(0,1)
    return noiseImg

In [5]:
def generateSPNoise(image,amplitude):
    """
    image:Mat(2D-array)
    amplitude:noise intensity(int)
    output:Mat(2D-array)
    """
    rows,cols=image.shape
    noiseImg=np.zeros(shape=image.shape,dtype=image.dtype)
    for i in range(rows):
        for j in range(cols):
            ran=random.uniform(0,1)
            if(ran>(1-amplitude)):
                noiseImg[i,j]=255
            elif(ran<amplitude):
                noiseImg[i,j]=0
            else:
                noiseImg[i,j]=image[i,j]
    return noiseImg

In [6]:
def boxFilter(image,boxsize):
    """
    image:Mat(2D-array)
    boxsize:odd integer(int)
    output:Mat(2D-array)
    """
    boxWidth=boxsize//2
    boxHeight=boxsize//2
    imagePad=np.pad(image,((boxWidth,boxWidth),(boxHeight,boxHeight)),'reflect')
    rows,cols=image.shape
    noiseImg=np.zeros(shape=image.shape,dtype=image.dtype)
    for i in range(rows):
        for j in range(cols):   
            noiseImg[i,j]=np.mean(imagePad[i:i+boxWidth,j:j+boxHeight])
    return noiseImg

In [7]:
def medianfilter(image,boxsize):
    """
    image:Mat(2D-array)
    boxsize:odd integer(int)
    output:Mat(2D-array)
    """
    boxWidth=boxsize//2
    boxHeight=boxsize//2
    imagePad=np.pad(image,((boxWidth,boxWidth),(boxHeight,boxHeight)),'reflect')
    rows,cols=image.shape
    noiseImg=np.zeros(shape=image.shape,dtype=image.dtype)
    for i in range(rows):
        for j in range(cols):
            seeds=imagePad[i:i+boxsize,j:j+boxsize].flatten()
            order=sorted(seeds)
            noiseImg[i,j]=order[boxsize*boxHeight+boxWidth]
    return noiseImg

## opening and closing

In [8]:
kernel=np.array([[0,1,1,1,0],
                [1,1,1,1,1],
                [1,1,1,1,1],
                [1,1,1,1,1],
                [0,1,1,1,0]])
kernelRow,kernelCol=kernel.shape
center=kernel[3,3]

In [9]:
def grayScaleDilation(src,kernel):
    dilationImg=np.zeros(shape=src.shape,dtype=src.dtype)
    dilateKernel=np.zeros(shape=kernel.shape,dtype=kernel.dtype)
    kernelHalf=(kernelCol-1)//2
    maximum=0
    for i in range(2,510):
        for j in range(2,510):
            for a in range(kernelRow):
                for b in range(kernelCol):
                    dilateKernel[a,b]=src[i-kernelHalf+a,j-kernelHalf+b]+kernel[a,b]
            dilationImg[i,j]=dilateKernel.max()
    return dilationImg

In [73]:

def grayScaleErosion(src,kernel):
    erosionImg=np.zeros(shape=src.shape,dtype=src.dtype)
    kernelHalf=(kernelCol-1)//2
    erosionKernel=np.zeros(shape=kernel.shape,dtype=src.dtype)
    global count,minimum
    for i in range(2,510):
        for j in range(2,510):
            count=0
            minimum=255
            for a in range(kernelRow):
                for b in range(kernelCol):
                    if(src[i-kernelHalf+a,j-kernelHalf+b]*kernel[a,b]!=0):
                        count=count+1
                        erosionKernel[a,b]=src[i-kernelHalf+a,j-kernelHalf+b]-kernel[a,b]
                        minimum=min(erosionKernel[a,b],minimum)
                    if(count==np.count_nonzero(kernel)):
                        erosionImg[i,j]=minimum
    return erosionImg

In [74]:
def grayscaleOpening(image,kernel):
    temp=grayScaleErosion(image,kernel)
    output=grayScaleDilation(temp,kernel)
    return output

In [75]:
def grayscaleClosing(image,kernel):
    temp=grayScaleDilation(image,kernel)
    output=grayScaleErosion(temp,kernel)
    return output

## SNR

In [5]:
def normalize(image):
    #pixelmax=255
    #pixelmin=0
    rows,cols=image.shape
    normalized=np.zeros(shape=image.shape)
    for i in range(rows):
        for j in range(cols):
            normalized[i,j]=(image[i,j])/(255)
    return normalized

In [6]:
def SNR(original,noise):
    import math
    vs=0
    u=0
    vn=0
    un=0
    #cv2.normalize(original,original, 0.0, 1.0, cv2.NORM_MINMAX)
    #cv2.normalize(noise,noise, 0.0, 1.0, cv2.NORM_MINMAX)
    rows,cols=original.shape
    #u
    for i in range (rows):
        for j in range(cols):
            u+=original[i,j]
    u=u/original.size
    #vs
    for i in range(rows):
        for j in range(cols):
            vs+=math.pow(original[i,j]-u,2)
    vs=vs/original.size
    #un
    for i in range(rows):
        for j in range(cols):
            un+= noise[i,j]-original[i,j]
    un=un/original.size
    #vn
    for i in range(rows):
        for j in range(cols):
             vn += math.pow(noise[i,j] - original[i,j] - un, 2)
    vn=vn/original.size
    #SNR
    temp=math.sqrt(vs)/math.sqrt(vn)
    return 20*math.log(temp,10)

In [7]:
#test snr
lena=cv2.imread("lena.bmp",0)
median=cv2.imread("median_5x5.bmp",0)
lean=normalize(lena)
median=normalize(median)
snr=SNR(lean,median)
snr


15.673697071703497

In [113]:
src.shape[1]

512

**1.box filter3*3**

**2.box filter 5*5**

**3.median filter 3*3**

**4.median filter 5*5**

**opening then closing**

**closing then opening**

//each with SNR//



### gaussian noise with amplitude=10

In [14]:
noiseG10=generateGaussianNoise(src,10)
cv2.imwrite("noiseG10.png",noiseG10)

In [16]:
box3=boxFilter(noiseG10,3)
cv2.imwrite("box3.png",box3)

In [19]:
box5=boxFilter(noiseG10,5)
#cv2.imwrite("box5.png",box5)

In [20]:
median3=medianfilter(noiseG10,3)
#cv2.imwrite("median3.png",median3)

In [21]:
median5=medianfilter(noiseG10,5)
#cv2.imwrite("median5.png",median5)

In [125]:
openThenClose=grayscaleOpening(noiseG10,kernel)
openThenClose=grayscaleClosing(openThenClose,kernel)
cv2.imwrite("openThenClose.png",openThenClose)

True

In [124]:
temp=grayscaleClosing(noiseG10,kernel)
closethenopen=grayscaleOpening(temp,kernel)
cv2.imwrite("closeThenOpen.png",closethenopen)

True

###  gaussian noise with amplitude=30

In [52]:
noiseG30=generateGaussianNoise(src,30)
cv2.imwrite("noiseG30.png",noiseG30)

True

In [53]:
box3=boxFilter(noiseG30,3)
cv2.imwrite("box3.png",box3)

True

In [54]:
box5=boxFilter(noiseG30,5)
cv2.imwrite("box5.png",box5)

True

In [55]:
median3=medianfilter(noiseG30,3)
cv2.imwrite("median3.png",median3)

True

In [56]:
median5=medianfilter(noiseG30,5)
cv2.imwrite("median5.png",median5)

True

In [39]:
openThenClose=grayscaleOpening(noiseG30,kernel)
openThenClose=grayscaleClosing(openThenClose,kernel)
cv2.imwrite("openThenClose.png",openThenClose)

True

In [40]:
temp=grayscaleClosing(noiseG30,kernel)
closethenopen=grayscaleOpening(temp,kernel)
cv2.imwrite("closeThenOpen.png",closethenopen)

True

### S&P noise with Prob. =0.1

In [57]:
noiseSP01=generateSPNoise(src,0.1)
cv2.imwrite("noiseSP01.png",noiseSP01)

True

In [58]:
box3=boxFilter(noiseSP01,3)
cv2.imwrite("box3.png",box3)

True

In [59]:
box5=boxFilter(noiseSP01,5)
cv2.imwrite("box5.png",box5)

True

In [60]:
median3=medianfilter(noiseSP01,3)
cv2.imwrite("median3.png",median3)

True

In [61]:
median5=medianfilter(noiseSP01,5)
cv2.imwrite("median5.png",median5)

True

In [62]:
openThenClose=grayscaleOpening(noiseSP01,kernel)
openThenClose=grayscaleClosing(openThenClose,kernel)
cv2.imwrite("openThenClose.png",openThenClose)

True

In [64]:
temp=grayscaleClosing(noiseSP01,kernel)
closethenopen=grayscaleOpening(temp,kernel)
cv2.imwrite("closeThenOpen.png",closethenopen)

True

### S&P noise with Prob. =0.05

In [65]:
noiseSP005=generateSPNoise(src,0.05)
cv2.imwrite("noiseSP005.png",noiseSP005)

True

In [66]:
box3=boxFilter(noiseSP005,3)
cv2.imwrite("box3.png",box3)

True

In [67]:
box5=boxFilter(noiseSP005,5)
cv2.imwrite("box5.png",box5)

True

In [68]:
median3=medianfilter(noiseSP005,3)
cv2.imwrite("median3.png",median3)

True

In [69]:
median5=medianfilter(noiseSP005,5)
cv2.imwrite("median5.png",median5)

True

In [70]:
openThenClose=grayscaleOpening(noiseSP005,kernel)
openThenClose=grayscaleClosing(openThenClose,kernel)
cv2.imwrite("openThenClose.png",openThenClose)

True

In [76]:
temp=grayscaleClosing(noiseSP005,kernel)
closethenopen=grayscaleOpening(temp,kernel)
cv2.imwrite("closeThenOpen.png",closethenopen)

True